In [1]:
import pandas as pd
from datetime import datetime
import tabula

# Importing public resources files

In [2]:
df_public_resources = pd.read_csv("public_resources_list.csv")

In [3]:
links_pax_entries_stations = df_public_resources.query(
    " file_type == 'csv' & resource_name.str.contains('Entrada de Passageiros por Estação - Média Dias Úteis') "
    )
links_pax_entries_stations

,resource_name,resource_link,file_type,mod_date
3,Entrada de Passageiros por Estação - Média Dia...,https://transparencia.metrosp.com.br/sites/def...,csv,2023-05-09
10,Entrada de Passageiros por Estação - Média Dia...,https://transparencia.metrosp.com.br/sites/def...,csv,2023-05-09
17,Entrada de Passageiros por Estação - Média Dia...,https://transparencia.metrosp.com.br/sites/def...,csv,2023-05-09


In [4]:
links_pax_entries_stations["resource_name"].to_list()

['Entrada de Passageiros por Estação - Média Dias Úteis - 2023',
 'Entrada de Passageiros por Estação - Média Dias Úteis - 2022',
 'Entrada de Passageiros por Estação - Média Dias Úteis - 2021']

In [5]:
# Mapping the months to its numeric equivalent
month_map = {
    'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4,
    'mai': 5, 'jun': 6, 'jul': 7, 'ago': 8,
    'set': 9, 'out': 10, 'nov': 11, 'dez': 12
}

# Complete PES wrangling for L01 - Blue

In [6]:
def l01_pes(url, year):
    add_skip = 0
    if year == "2022":
        add_skip = -98
    # Read the CSV file
    df_raw = pd.read_csv(
        url, 
        encoding="latin-1", 
        sep=";",
        skiprows=5,
        skipfooter=166+add_skip,
        usecols=range(0, 13),
        engine='python',
        )
    # Reshape the DataFrame using melt    
    df_raw = df_raw.melt(id_vars=["Estação"], var_name="month", value_name="dpea")
    # Remove "*" from the month column
    df_raw["month"] = df_raw["month"].str.replace(r"\*", "", regex=True)
    # Convert month names to datetime objects
    df_raw["month"] = df_raw["month"].apply(lambda x: pd.to_datetime(year + str(month_map[x.lower()]), format="%Y%m"))
    # Rename the "Estação" column to "station"
    df_raw.rename(columns={"Estação": "station"}, inplace=True)
    df_raw['station'] = df_raw['station'].str.replace(r'[¹²2]', '', regex=True)
    # Add a "line" column with value 1
    df_raw["line"] = 1
    # Reorder the columns
    df_raw = df_raw.reindex(columns=["month", "line", "station", "dpea"])
    # converting dpea to numeric and multiplying by 1_000 to show unitary data
    df_raw["dpea"] = pd.to_numeric(df_raw["dpea"], errors="coerce")
    df_raw["dpea"].fillna(0, inplace=True)
    df_raw["dpea"] = df_raw["dpea"] * 1_000
    
    return df_raw

In [7]:
results_l01_pes = []
for index, row in links_pax_entries_stations.iterrows():
    resource_link = row["resource_link"]
    year = row["resource_name"].split(" - ")[-1]
    df_processed = l01_pes(resource_link, year)
    results_l01_pes.append(df_processed)
    

In [8]:
l01_pes_complete = pd.concat(results_l01_pes, ignore_index=True).sort_values("month")
l01_pes_complete

,month,line,station,dpea
573,2021-01-01,1,Parada Inglesa,7000.0
572,2021-01-01,1,Jardim São Paulo-Ayrton Senna,6000.0
571,2021-01-01,1,Santana,33000.0
570,2021-01-01,1,Carandiru,6000.0
569,2021-01-01,1,Portuguesa-Tietê,31000.0
...,...,...,...,...
255,2023-12-01,1,São Judas,0.0
254,2023-12-01,1,Conceição,0.0
253,2023-12-01,1,Jabaquara,0.0
265,2023-12-01,1,Sé,0.0


In [9]:
l01_pes_complete.to_csv("final_datasets/l01_pes_complete.csv", index=False)

# Complete PES wrangling for L02 - Green

In [10]:
def l02_pes(url, year):
    add_skip = 0
    if year == "2022":
        add_skip = -98
    df_raw = pd.read_csv(
        url, 
        encoding="latin-1", 
        sep=";",
        skiprows=35,
        skipfooter=145 + add_skip,
        usecols=range(0, 13),
        engine='python',
        )
    df_raw = df_raw.melt(id_vars=["Estação"], var_name="month", value_name="dpea")
    df_raw["month"] = df_raw["month"].str.replace(r"\*", "", regex=True)
    df_raw["month"] = df_raw["month"].apply(lambda x: pd.to_datetime(year + str(month_map[x.lower()]), format="%Y%m"))
    df_raw.rename(columns={"Estação": "station"}, inplace=True)
    # Remove ¹ and ² characters
    df_raw['station'] = df_raw['station'].str.replace(r'[¹²]', '', regex=True)
    df_raw["line"] = 2
    df_raw = df_raw.reindex(columns=["month", "line", "station", "dpea"])
    # converting dpea to numeric and multiplying by 1_000 to show unitary data
    df_raw["dpea"] = pd.to_numeric(df_raw["dpea"], errors="coerce")
    df_raw["dpea"].fillna(0, inplace=True)
    df_raw["dpea"] = df_raw["dpea"] * 1_000
    
    return df_raw

In [11]:
results_l02_pes = []
for index, row in links_pax_entries_stations.iterrows():
    resource_link = row["resource_link"]
    year = row["resource_name"].split(" - ")[-1]
    df_processed = l02_pes(resource_link, year)
    results_l02_pes.append(df_processed)

In [12]:
l02_pes_complete = pd.concat(results_l02_pes, ignore_index=True).sort_values("month")
l02_pes_complete

,month,line,station,dpea
348,2021-01-01,2,Santuário N.S. de Fátima-Sumaré,5000.0
347,2021-01-01,2,Clínicas,13000.0
346,2021-01-01,2,Consolação,49000.0
345,2021-01-01,2,Trianon-Masp,21000.0
344,2021-01-01,2,Brigadeiro,25000.0
...,...,...,...,...
156,2023-12-01,2,Sacomã,0.0
155,2023-12-01,2,Tamanduateí,0.0
154,2023-12-01,2,Vila Prudente,0.0
160,2023-12-01,2,Ana Rosa,0.0


In [13]:
l02_pes_complete.to_csv("final_datasets/l02_pes_complete.csv", index=False)

# Complete PES wrangling for L03 - red

In [14]:
def l03_pes(url, year):
    add_skip = 0
    if year == "2022":
        add_skip = -98
    df_raw = pd.read_csv(
        url, 
        encoding="latin-1", 
        sep=";",
        skiprows=56,
        skipfooter=120 + add_skip,
        usecols=range(0, 13),
        engine='python',
        )
    df_raw = df_raw.melt(id_vars=["Estação"], var_name="month", value_name="dpea")
    df_raw["month"] = df_raw["month"].str.replace(r"\*", "", regex=True)
    df_raw["month"] = df_raw["month"].apply(lambda x: pd.to_datetime(year + str(month_map[x.lower()]), format="%Y%m"))
    df_raw.rename(columns={"Estação": "station"}, inplace=True)
    # Remove ¹ and ² characters
    df_raw['station'] = df_raw['station'].str.replace(r'[¹²]', '', regex=True)
    df_raw["line"] = 3
    df_raw = df_raw.reindex(columns=["month", "line", "station", "dpea"])
    # converting dpea to numeric and multiplying by 1_000 to show unitary data
    df_raw["dpea"] = pd.to_numeric(df_raw["dpea"], errors="coerce")
    df_raw["dpea"].fillna(0, inplace=True)
    df_raw["dpea"] = df_raw["dpea"] * 1_000
    
    return df_raw

In [15]:
results_l03_pes = []
for index, row in links_pax_entries_stations.iterrows():
    resource_link = row["resource_link"]
    year = row["resource_name"].split(" - ")[-1]
    df_processed = l03_pes(resource_link, year)
    results_l03_pes.append(df_processed)

In [16]:
l03_pes_complete = pd.concat(results_l03_pes, ignore_index=True).sort_values("month")
l03_pes_complete

,month,line,station,dpea
448,2021-01-01,3,Marechal Deodoro,18000.0
447,2021-01-01,3,Santa Cecília,16000.0
446,2021-01-01,3,República,72000.0
445,2021-01-01,3,Anhangabaú,32000.0
444,2021-01-01,3,Sé,123000.0
...,...,...,...,...
200,2023-12-01,3,Patriarca,0.0
199,2023-12-01,3,Artur Alvim,0.0
198,2023-12-01,3,Corinthians-Itaquera,0.0
207,2023-12-01,3,Bresser-Moóca,0.0


In [17]:
l03_pes_complete.to_csv("final_datasets/l03_pes_complete.csv", index=False)

# Complete PES wrangling for L15 - silver

In [18]:
def l15_pes(url, year):
    add_skip = 0
    if year == "2022":
        add_skip = -98
    df_raw = pd.read_csv(
        url, 
        encoding="latin-1", 
        sep=";",
        skiprows=80,
        skipfooter=103 + add_skip,
        usecols=range(0, 13),
        engine='python',
        )
    df_raw = df_raw.melt(id_vars=["Estação"], var_name="month", value_name="dpea")
    df_raw["month"] = df_raw["month"].str.replace(r"\*", "", regex=True)
    df_raw["month"] = df_raw["month"].apply(lambda x: pd.to_datetime(year + str(month_map[x.lower()]), format="%Y%m"))
    df_raw.rename(columns={"Estação": "station"}, inplace=True)
    # Remove ¹ and ² characters
    df_raw['station'] = df_raw['station'].str.replace(r'[¹²]', '', regex=True)
    df_raw["line"] = 15
    df_raw = df_raw.reindex(columns=["month", "line", "station", "dpea"])
    # converting dpea to numeric and multiplying by 1_000 to show unitary data
    df_raw["dpea"] = pd.to_numeric(df_raw["dpea"], errors="coerce")
    df_raw["dpea"].fillna(0, inplace=True)
    df_raw["dpea"] = df_raw["dpea"] * 1_000
    
    return df_raw

In [19]:
results_l15_pes = []
for index, row in links_pax_entries_stations.iterrows():
    resource_link = row["resource_link"]
    year = row["resource_name"].split(" - ")[-1]
    df_processed = l15_pes(resource_link, year)
    results_l15_pes.append(df_processed)

In [20]:
l15_pes_complete = pd.concat(results_l15_pes, ignore_index=True).sort_values("month")
l15_pes_complete

,month,line,station,dpea
273,2021-01-01,15,São Mateus,11000.0
272,2021-01-01,15,Fazenda da Juta,2000.0
271,2021-01-01,15,Sapopemba,5000.0
270,2021-01-01,15,Jardim Planalto,2000.0
269,2021-01-01,15,Vila União,3000.0
...,...,...,...,...
127,2023-12-01,15,Jardim Planalto,0.0
128,2023-12-01,15,Sapopemba,0.0
129,2023-12-01,15,Fazenda da Juta,0.0
130,2023-12-01,15,São Mateus,0.0


In [21]:
l15_pes_complete.to_csv("final_datasets/l15_pes_complete.csv", index=False)

# Importing private resources files

In [22]:
df_private_resources = pd.read_csv("private_resources_list.csv")
df_private_resources

,resource_name,resource_link,file_type,line
0,Entrada de passageiros por estação - Abril 2023,https://www.viaquatro.com.br/Media/ContentPage...,pdf,4
1,Entrada de passageiros por estação - Março 2023,https://www.viaquatro.com.br/Media/ContentPage...,pdf,4
2,Entrada de passageiros por estação - Fevereiro...,https://www.viaquatro.com.br/Media/ContentPage...,pdf,4
3,Entrada de passageiros por estação - Janeiro 2023,https://www.viaquatro.com.br/Media/ContentPage...,pdf,4
4,Entrada de passageiros por estação - Dezembro ...,https://www.viaquatro.com.br/Media/ContentPage...,pdf,4
...,...,...,...,...
385,Entrada de passageiros pelas linhas de bloquei...,https://www.viamobilidade.com.br/assets/viamob...,pdf,5
386,Passageiros transportados - Abril de 2023 Down...,https://www.viamobilidade.com.br/assets/viamob...,pdf,5
387,Passageiros transportados - Março de 2023 Down...,https://www.viamobilidade.com.br/assets/viamob...,pdf,5
388,Passageiros transportados - Fevereiro de 2023 ...,https://www.viamobilidade.com.br/assets/viamob...,pdf,5


In [33]:
links_line4_pes = df_private_resources.query(
    "resource_name.str.contains('Entrada de passageiros por estação -')"
    "& ~resource_name.str.contains('Entrada de passageiros por estação - [Mm]')"
    "& ~resource_name.str.contains('2020')"
    )

In [97]:
links_line5_pes = df_private_resources.query(
    "resource_name.str.contains('Entrada de passageiros por estação -')"
    "& resource_name.str.contains('(2021|2022|2023)')"
    "& line == 5"
)

C:\Users\rafae\AppData\Local\Temp\ipykernel_6128\3095968290.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  links_line5_pes = df_private_resources.query(


# Complete wrangling for L04 - Yellow

In [93]:
def l04_pes(url, date_raw):
    # Read the PDF into a DataFrame
    df_raw = tabula.read_pdf(url, pages="all")[0]
    # Skip the first and last rows of the DataFrame
    df_raw = df_raw.iloc[1:-1]
    # Extract the year and month from the date_raw string
    year = date_raw.split(" ")[-1]
    month = date_raw.split(" ")[0]
    # Convert the month abbreviation to a datetime object
    df_raw["month"] = pd.to_datetime(str(year) + str(month_map[month.lower()[:3]]), format="%Y%m")
    # Replace comma with period and convert the column to numeric values
    df_raw["Unnamed: 0"] = pd.to_numeric(df_raw["Unnamed: 0"].str.replace(",", "."), errors="coerce")
    # Replace missing values with 0 in the column
    df_raw["Unnamed: 0"].fillna(0, inplace=True)
    # Multiply the column by 1000
    df_raw["Unnamed: 0"] = df_raw["Unnamed: 0"] * 1_000
    # Rename columns and remove special characters from the 'station' column
    df_raw.rename(columns={"Linha 4 - Amarela": "station", "Unnamed: 0": "dpea"}, inplace=True)
    df_raw['station'] = df_raw['station'].str.replace(r'[¹²³1234]', '', regex=True)
    # Add the line column and reorder the columns
    df_raw["line"] = 4
    df_raw = df_raw.reindex(columns=["month", "line", "station", "dpea"])

    return df_raw

In [94]:
results_l04_pes = []
for index, row in links_line4_pes.iterrows():
    resource_link = row["resource_link"]
    year = row["resource_name"].split(" - ")[-1]
    df_processed = l04_pes(resource_link, year)
    results_l04_pes.append(df_processed)

In [95]:
l04_pes_complete = pd.concat(results_l04_pes, ignore_index=True).sort_values("month")
l04_pes_complete

,month,line,station,dpea
239,2021-01-01,4,Luz,75000.0
230,2021-01-01,4,São Paulo - Morumbi,26000.0
231,2021-01-01,4,Butantã,22000.0
232,2021-01-01,4,Pinheiros,55000.0
233,2021-01-01,4,Faria Lima,16000.0
...,...,...,...,...
8,2023-04-01,4,Higienópolis - Mackenzie,27480.0
9,2023-04-01,4,República,106200.0
10,2023-04-01,4,Luz,135690.0
5,2023-04-01,4,Fradique Coutinho,17180.0


In [115]:
l04_pes_complete.to_csv("final_datasets/l04_pes_complete.csv", index=False)

# Complete wrangling for L05 - Purple

In [112]:
def l05_pes(url, date_raw):
    # Read the PDF into a DataFrame
    df_raw = tabula.read_pdf(url, pages="all")[0]
    # Skip the first and last rows of the DataFrame
    df_raw = df_raw.iloc[1:-1]
    # Extract the year and month from the date_raw string
    year = date_raw.split(" ")[2]
    month = date_raw.split(" ")[0]
    # Convert the month abbreviation to a datetime object
    df_raw["month"] = pd.to_datetime(str(year) + str(month_map[month.lower()[:3]]), format="%Y%m")
    # Replace comma with period and convert the column to numeric values
    df_raw["Unnamed: 0"] = pd.to_numeric(df_raw["Unnamed: 0"].str.replace(",", "."), errors="coerce")
    # Replace missing values with 0 in the column
    df_raw["Unnamed: 0"].fillna(0, inplace=True)
    # Multiply the column by 1000
    df_raw["Unnamed: 0"] = df_raw["Unnamed: 0"] * 1_000
    # Rename columns and remove special characters from the 'station' column
    df_raw.rename(columns={"Linha 5 - Lilás": "station", "Unnamed: 0": "dpea"}, inplace=True)
    df_raw['station'] = df_raw['station'].str.replace(r'[¹²³1234]', '', regex=True)
    # Add the line column and reorder the columns
    df_raw["line"] = 5
    df_raw = df_raw.reindex(columns=["month", "line", "station", "dpea"])

    return df_raw

In [113]:
results_l05_pes = []
for index, row in links_line5_pes.iterrows():
    resource_link = row["resource_link"]
    year = row["resource_name"].split(" - ")[-1]
    df_processed = l05_pes(resource_link, year)
    results_l05_pes.append(df_processed)

In [114]:
l05_pes_complete = pd.concat(results_l05_pes, ignore_index=True).sort_values("month")
l05_pes_complete

,month,line,station,dpea
187,2021-01-01,5,Capão Redondo,63000.0
189,2021-01-01,5,Vila das Belezas,9000.0
190,2021-01-01,5,Giovanni Gronchi,15000.0
191,2021-01-01,5,Santo Amaro,56000.0
192,2021-01-01,5,Largo Treze,21000.0
...,...,...,...,...
420,2023-04-01,5,Moema,19680.0
421,2023-04-01,5,AACD - Servidor,11580.0
424,2023-04-01,5,Chácara Klabin,63710.0
415,2023-04-01,5,Alto da Boa Vista,5710.0


In [116]:
l05_pes_complete.to_csv("final_datasets/l05_pes_complete.csv", index=False)